# 1. Загрузка и подготовка данных

Отзывы для обучения уже загружены скриптом в SQLite3 бд.

In [11]:
import pandas as pd
import numpy as np
import sqlite3
import html
%matplotlib inline

In [3]:
conn = sqlite3.connect('reviews_395.db')
c = conn.cursor()

In [4]:
train_data = list(c.execute("SELECT * FROM reviews;"))
conn.close()

In [12]:
data_raw = pd.read_csv("./train_data.csv")

In [17]:
data_asessed_1 = pd.read_csv("./train_data_results_1.csv", header=None).drop(columns=[0]).rename(columns={1:"text", 2: "label"})
data_asessed_2 = pd.read_csv("./train_data_results_2.csv", header=None).drop(columns=[0]).rename(columns={1:"text", 2: "label"})

In [14]:
data_assessed = data_asessed_1.append(data_asessed_2)

In [19]:
data_raw.text[1] in data_assessed["text"].unique()

True

In [27]:
len(data_assessed)

1591

In [24]:
uqs = data_assessed.text.unique()
data_raw["duplicate"] = data_raw["text"].apply(lambda x: x in uqs)

In [31]:
data_assessed.columns

Index(['text', 'label'], dtype='object')

In [33]:
data.drop(columns=["Unnamed: 0","duplicate"], inplace=True)

In [32]:
data = data_assessed.append(data_raw[data_raw.duplicate == False])

## 1.1 Очистка.

В данные попало много лишних тегов и прочего мусора. Для начала я уберу его, а также нормализую пунктуацию.

In [35]:
import re

In [45]:
re.sub(r"[\s.,-]"," ", "...   123 не - akjsdj \n")
re.sub(r"\s+"," ", "...   123 не - akjsdj \n")

'... 123 не - akjsdj '

In [47]:
def normalize_text(text):
    _t = html.unescape(text)
    _t = _t.replace("<p>"," ")
    _t = _t.replace("</p>", " ")
    _t = _t.replace("\n", " ")
    _t = _t.replace("\r", " ")
    _t = _t.replace("\t", " ")
    _t = _t.replace('"', " ")
    return _t.strip()

In [46]:
def normalize_text_re(text):
    _t = text
    _t = re.sub(r"[\s.,-><;:!?()]", " ", _t)
    _t = re.sub(r"\s+", " ", _t)
    return _t

In [48]:
data["text"] = data["text"].apply(lambda x: normalize_text(x))
data["text"] = data["text"].apply(lambda x: normalize_text_re(x))

In [49]:
len(data["text"].unique())

10098

In [50]:
data["text"].tail(30)

10122    Плюсы достоинство что это айфон уже Минусы зар...
10123    Плюсы Удобство качество агрегата простота в ис...
10124    Плюсы экран скорость работы внешний вид размер...
10125    Плюсы Аппарат показал себя на лучшем уровне У ...
10126    Плюсы Простота и удобство и даже тунец работае...
10127    Плюсы У меня было blackberry Все были на BB OS...
10128    Плюсы качественная сборка качественное ПО Мину...
10129    Плюсы лет а он все также работает Минусы нет В...
10130    Плюсы флеш карту принимает удобный лёгкий инте...
10131    Плюсы Качество связи Динамик и разговорный и в...
10132    Плюсы маленькии уже есть опера это хорошо аськ...
10133    Плюсы мощный приём передача практически всё ес...
10134    Плюсы большие кнопки громкий звук фонарик длит...
10135    Плюсы Удобный приятный лёгкий Большие клавиши ...
10136    Плюсы сим карты крупные цифры голосовой набор ...
10137    Плюсы большой экран цена гнезда для наушников ...
10138    Плюсы Нет люфта клавиши нажимаются плавно Мину.

In [51]:
data.to_csv("./train_data_consolidated.csv")

## 2. Векторизация

Я хочу воспользоваться библиотекой gensim, потому что я уже использовал их эмбеддинги, и они отлично показали себя в классификации, даже на плохо обработанном датасете.

Чтобы потом удобно запаковать это в sklearn pipeline, я реализую свой класс по образу `TfIdfVectorizer` из `sklearn.feature_extraction`

In [52]:
import gensim
from gensim.utils import simple_preprocess
from gensim.models import doc2vec
from tqdm import tqdm
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import _VectorizerMixin
import numpy as np

class VectorizerTransformer(_VectorizerMixin, BaseEstimator):
    def __init__(self):
        pass

    def fit(self, raw_documents, y=None):
        X = self.preprocess(raw_documents)
        # print("Creating model...")
        model = gensim.models.doc2vec.Doc2Vec(
            vector_size=60, 
            min_count=10,
            epochs=40
        )
        # print("Building vocab...")
        model.build_vocab(X)
        # print("Training doc2vec...")
        model.train(X, total_examples=model.corpus_count, epochs=model.epochs)
        self.model = model
        return self

    def transform(self, raw_documents):
        X = self.preprocess(raw_documents)
        # print("Iinferring vectors...")
        vectorized_texts = []
        for doc_id, _ in enumerate(tqdm(X, desc="Inferring vectors: ")):
            inferred_vector = self.model.infer_vector(X[doc_id].words)
            vectorized_texts.append(inferred_vector)

        return vectorized_texts

    def preprocess(self, raw_documents):
        # print("Tokenization...")
        processed_texts = []
        for idx, text in enumerate(tqdm(raw_documents, desc="Tokenization: ")):
            text = normalize_text(text)
            text = normalize_text_re(text)
            processed_texts.append(doc2vec.TaggedDocument(simple_preprocess(text), [idx]))
        return processed_texts


    def fit_transform(self, texts, y=None) -> np.ndarray:
        self.fit(texts)
        X = self.transform(texts)
        
        return np.array(X)

## 3. Обучение модели 

In [53]:
X = data["text"]
# y = data.label.map({"pos": 1, "neg": 0})
y = data.label

In [54]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, make_scorer
from sklearn.model_selection import cross_val_score, cross_validate

_model = make_pipeline(VectorizerTransformer(), GradientBoostingClassifier())



In [55]:
cross_val_score(_model, X, y, n_jobs=6)

array([0.92072871, 0.91674877, 0.92019704, 0.91724138, 0.91674877])

In [76]:
_model.fit(X, y)

Inferring vectors: 100%|██████████| 1591/1591 [00:02<00:00, 581.03it/s]


Pipeline(steps=[('vectorizertransformer', VectorizerTransformer()),
                ('gradientboostingclassifier', GradientBoostingClassifier())])

В принципе, качество уже не такое гадкое, можно попробовать сделать сабмит.

## 4. Улучшение модели

Улучшить модель можно несколькими путями:
+ 1. Улучшить препроцессинг текста
+ 2. Подобрать параметры модели векторизации
+ 3. Подобрать параметры классификатора
+ 4. Попробовать другие классификаторы
+ 5. Сбалансировать классы для обучения

Я сделаю только п.3 и 5

In [56]:
from sklearn.model_selection import GridSearchCV, ParameterGrid
_acc = make_scorer(accuracy_score)
_roc = make_scorer(roc_auc_score)
param_grid = {
    "n_estimators":[100,200,500,700],
    "criterion": ("friedman_mse", "mse", "mae"),
    "max_depth": [3,4,5]
}


In [57]:
# так как я не подбираю параметры для векторизатора, 
# я сразу обучу его на полной выборке, потому что именно в таком виде он пойдёт в прод
_vectorizer = VectorizerTransformer()
vecs = _vectorizer.fit_transform(X)


Inferring vectors: 100%|██████████| 10151/10151 [00:21<00:00, 478.82it/s]


In [110]:
vecs = _vectorizer.transform(X)

Inferring vectors: 100%|██████████| 10152/10152 [00:21<00:00, 475.87it/s]


In [58]:
# мало!
sum(y == 0)

789

In [109]:
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import ADASYN
vecs_us, y_us = NearMiss(version=3).fit_resample(vecs, y)
# vecs_us, y_us = ADASYN(n_jobs=6).fit_resample(vecs, y)

In [110]:
len(y_us)

1578

In [95]:
%%time
gscv_ = GridSearchCV(
    estimator=GradientBoostingClassifier(n_iter_no_change=20),
    param_grid=param_grid, 
    scoring={"accuracy":_acc,"roc_auc": _roc}, 
    n_jobs=4, refit=False, cv=4)
res = gscv_.fit(vecs_us, y_us)

CPU times: user 238 ms, sys: 60.3 ms, total: 298 ms
Wall time: 3min 10s


In [96]:
pd.DataFrame(res.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_n_estimators,params,split0_test_accuracy,split1_test_accuracy,...,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,split3_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc
0,1.393110,0.006100,0.001924,0.000101,friedman_mse,3,100,"{'criterion': 'friedman_mse', 'max_depth': 3, ...",0.865823,0.827848,...,0.838392,0.016261,8,0.865931,0.827616,0.835025,0.824873,0.838361,0.016345,8
1,2.070721,0.438126,0.001951,0.000215,friedman_mse,3,200,"{'criterion': 'friedman_mse', 'max_depth': 3, ...",0.868354,0.835443,...,0.846000,0.014113,2,0.868482,0.835230,0.832487,0.847716,0.845979,0.014203,2
2,2.310287,0.514617,0.002065,0.000286,friedman_mse,3,500,"{'criterion': 'friedman_mse', 'max_depth': 3, ...",0.860759,0.825316,...,0.844742,0.012930,4,0.860932,0.825091,0.850254,0.842640,0.844729,0.013068,4
3,1.887819,0.570442,0.001888,0.000281,friedman_mse,3,1000,"{'criterion': 'friedman_mse', 'max_depth': 3, ...",0.870886,0.815190,...,0.839032,0.021339,7,0.871007,0.814952,0.845178,0.824873,0.839002,0.021451,7
4,1.982360,0.291326,0.002040,0.000090,friedman_mse,5,100,"{'criterion': 'friedman_mse', 'max_depth': 5, ...",0.835443,0.830380,...,0.823182,0.009934,15,0.835512,0.830129,0.812183,0.814721,0.823136,0.009910,15
5,2.161622,0.124025,0.002098,0.000048,friedman_mse,5,200,"{'criterion': 'friedman_mse', 'max_depth': 5, ...",0.865823,0.815190,...,0.828248,0.022587,10,0.865969,0.814990,0.824873,0.807107,0.828235,0.022677,10
6,2.213941,0.448726,0.002099,0.000206,friedman_mse,5,500,"{'criterion': 'friedman_mse', 'max_depth': 5, ...",0.845570,0.817722,...,0.826356,0.013789,12,0.845703,0.817515,0.832487,0.809645,0.826338,0.013868,12
7,2.662204,0.359771,0.002303,0.000103,friedman_mse,5,1000,"{'criterion': 'friedman_mse', 'max_depth': 5, ...",0.868354,0.817722,...,0.827610,0.025278,11,0.868482,0.817502,0.799492,0.824873,0.827587,0.025351,11
8,1.983237,0.388090,0.002109,0.000229,friedman_mse,7,100,"{'criterion': 'friedman_mse', 'max_depth': 7, ...",0.837975,0.789873,...,0.811784,0.017512,21,0.838051,0.789673,0.814721,0.804569,0.811753,0.017603,21
9,1.560334,0.276050,0.001893,0.000155,friedman_mse,7,200,"{'criterion': 'friedman_mse', 'max_depth': 7, ...",0.830380,0.777215,...,0.802279,0.019876,23,0.830500,0.776957,0.791878,0.809645,0.802245,0.020000,23


Лучший набор параметров:


In [39]:
{'criterion': 'friedman_mse', 'max_depth': 3, 'n_estimators':200}
# Дефолтные, кроме количества estimators.

{'criterion': 'friedman_mse', 'max_depth': 3, 'n_estimators': 200}

In [81]:
model_final = make_pipeline(VectorizerTransformer(), GradientBoostingClassifier(criterion="friedman_mse", n_estimators=100, n_iter_no_change=20))
model_final.fit(X, y)

Inferring vectors: 100%|██████████| 1591/1591 [00:02<00:00, 594.93it/s]


Pipeline(steps=[('vectorizertransformer', VectorizerTransformer()),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(criterion='mse', n_estimators=500,
                                            n_iter_no_change=20))])

In [87]:
from sklearn.svm import SVC
model_final = make_pipeline(SVC())

print(cross_validate(model_final, vecs_us, y_us, n_jobs=3,cv=3, scoring=("accuracy", "roc_auc")))
# model_final.fit(X, y)

{'fit_time': array([0.02243423, 0.01695013, 0.01814723]), 'score_time': array([0.01682401, 0.0126977 , 0.01354456]), 'test_accuracy': array([0.9486692 , 0.88403042, 0.94676806]), 'test_roc_auc': array([0.99632783, 0.93317816, 0.95537018])}


In [111]:
# from sklearn.svm import Gradi
model_final = make_pipeline(GradientBoostingClassifier(criterion="friedman_mse", n_estimators=100, n_iter_no_change=20))

print(cross_validate(model_final, vecs_us, y_us, n_jobs=6, scoring=("accuracy", "roc_auc"), cv=3))

{'fit_time': array([1.2148211 , 0.34953213, 0.43612862]), 'score_time': array([0.00292349, 0.00183368, 0.00198507]), 'test_accuracy': array([0.45627376, 0.60646388, 0.59125475]), 'test_roc_auc': array([0.50534199, 0.61079385, 0.64248435])}


In [112]:
model_final.fit(vecs_us, y_us)

Pipeline(steps=[('gradientboostingclassifier',
                 GradientBoostingClassifier(n_iter_no_change=20))])

## 5. Инференс и подготовка сабмита

In [65]:
import bs4
test = []
with open("test.csv") as tfile:
    sp = bs4.BeautifulSoup(tfile)
    revs = sp.findAll("review")
    for r in revs:
        test.append(r.text)

# pd.read_csv("test.csv")

In [9]:
test

,Id,text,y
0,0,"Ужасно слабый аккумулятор, это основной минус ...",0
1,1,ценанадежность-неубиваемостьдолго держит батар...,1
2,2,"подробнее в комментариях\nК сожалению, факт по...",0
3,3,я любительница громкой музыки. Тише телефона у...,0
4,4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех...",1
...,...,...,...
95,95,"Нет передней камеры, внутренняя память очень м...",0
96,96,"Звук при прослушивание музыки хороший,не глючи...",1
97,97,Очень маленькая память забита вшитыми и соверш...,0
98,98,"Удобный корпус,стандартное меню нокиа,камера д...",1


In [104]:
test = pd.read_csv("./test_.csv")

In [105]:
test.head()

,Id,text,y
0,0,"Ужасно слабый аккумулятор, это основной минус ...",0
1,1,ценанадежность-неубиваемостьдолго держит батар...,1
2,2,"подробнее в комментариях\nК сожалению, факт по...",0
3,3,я любительница громкой музыки. Тише телефона у...,0
4,4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех...",1


In [113]:
test["prediction"] = model_final.predict(_vectorizer.transform(test.text))

Inferring vectors: 100%|██████████| 100/100 [00:00<00:00, 266.38it/s]


neg    100
Name: y, dtype: int64

In [115]:
accuracy_score(test["y"], test["prediction"])

0.49

In [ ]:
test["y"] = test.y.map({1: "pos", 0: "neg"})
submission["y"].value_counts()

In [106]:
submission.to_csv("./submission.csv", columns=["y"], index_label="Id")

In [7]:
submission.to_csv("./test_.csv", index_label="Id", columns=["text"],quoting=csv.QUOTE_NONNUMERIC )

In [ ]:
from csv import 

In [4]:
import csv

## 6. Упаковка модели

In [ ]:
import pickle as pkl

In [120]:
csv.

<module 'csv' from '/home/master/.local/share/miniconda3/lib/python3.8/csv.py'>